In [1]:
import torch
from d2l import torch as d2l

多输入通道的互相关运算

In [8]:
# 实现二维互相关运算
# X：输入矩阵，K：核矩阵，Y：输出矩阵
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [19]:
# 实现多输入通道的2D互相关运算
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [20]:
X = torch.tensor([
    [
        [0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]
    ],
    [
        [1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]
    ]
])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0 ,2.0], [3.0, 4.0]]])

In [21]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

In [22]:
# X是2D，K是4D
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [23]:
# 从三通道K创建四通道K，第1，2，3维度分别是K，K+1，K+2，然后在0维度上stack起来
K = torch.stack((K, K+1, K+2), 0)
K, K.shape

(tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]),
 torch.Size([3, 2, 2, 2]))

In [24]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1×1 卷积

In [25]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [26]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [27]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1-Y2).sum()) < 1e-6